In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import os
import random

In [ ]:
binary_dir = '../embedded_binaries/'
files = os.listdir(binary_dir)
nps = [x for x in files if x.endswith('.npy')]
fns = [x for x in files if x.endswith('.txt')]

results_dir = '../yara/results/'
result_files = os.listdir(results_dir)

In [ ]:
def get_top_N(target_file, n, filenames, numpy_files):
    min_bhatt = 1
    top_n = [(1, 'NULL')]
    for np_, fn_ in zip(numpy_files, filenames):
        compare_batch = np.load(binary_dir + np_)
        compare_batch_fn = [x.strip('\n') for x in open(binary_dir + fn_, "r")]
        total_bhatt = 0
        for compare_file, compare_fn in zip(compare_batch, compare_batch_fn):
            total_p = 0
            for em_dim in range(256):
                m1 = target_file[em_dim][0]
                std1 = target_file[em_dim][1]
                m2 = compare_file[em_dim][0]
                std2 = compare_file[em_dim][1]

                var_a = std1**2
                var_b = std2**2

                comp1 = np.log((var_a / var_b + var_b / var_a + 2) / 4) / 4
                comp2 = (((m1 - m2)**2) / (var_a + var_b)) / 4
                bhatt = comp1 + comp2
                total_bhatt += bhatt
            average_bhatt = total_bhatt / 256
            if average_bhatt < min_bhatt and average_bhatt > 0:
                min_bhatt = average_bhatt
            top_n_vals = [x[0] for x in top_n]
            min_top_n = max(top_n_vals)
            min_top_n_idx = top_n_vals.index(max(top_n_vals))
            if average_bhatt < min_top_n and average_bhatt > 0:
                if len(top_n) < n:
                    top_n.append((average_bhatt, compare_fn))
                else:
                    top_n.pop(min_top_n_idx)
                    top_n.append((average_bhatt, compare_fn))
    return pd.DataFrame(top_n).sort_values(by=0)

In [ ]:
rules_results = []
with open('/Users/malware/Downloads/results/antidebug_rules_results.txt') as f:
    for line in f.readlines():
        rules, results = line.split()
        result = results.split('/')[-1]
        rules_results.append((rules, result))

In [ ]:
rr_df = pd.DataFrame(rules_results)

In [ ]:
top_ns = {}
for match in rr_df[rr_df[0]=='SEH__vba'][1].values:
    target_file_name = match
    for fn in fns:
        target_file_idx = 0
        for x in open("../embedded_binaries/"+fn, "r"):
            if x.strip('\n') == target_file_name:
                idx = int(fn.split('batch_p_')[1].split('_')[0])
                target_batch = np.load('../embedded_binaries/batch_p_{}.npy'.format(idx))
                target_file = target_batch[target_file_idx]
            target_file_idx += 1
    top_n = get_top_N(target_file, 10, fns, nps)
    for tn in top_n[1].values:
        if tn in top_ns:
            top_ns[tn] += 1
        else:
            top_ns[tn] = 1
    print(top_ns)

In [ ]:
for i in rr_df[rr_df[0]=='hikit_xor_decode'][1].values:
    if i not in top_ns.keys():
        print(i)